# Causal Machine Learning Tutorial

This notebook compares targeted maximum likelihood estimation implemented via three different methods: a logistic regression, a boosted decision tree and a superlearner

In [7]:
import sys
import os

# Add src to the Python path
sys.path.append(os.path.abspath('../src'))

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import expit 

In [9]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC

In [11]:
from tutorial import generate_data, produce_dag, generate_data_nonlinearity

In [10]:
import statsmodels.api as sm
import statsmodels.formula.api as smf # allows R-like syntax 

In [12]:
from tmle import targeting_step

In [13]:
from doubleml import DoubleMLData
from doubleml import DoubleMLIRM

This comparison uses the same dataset as the `tmle_walkthrough.ipynb` notebook. We can change the data generating function to make it a little bit less well behaved!

In [2]:
df = generate_data_nonlinearity(2000000, 555) # generate a huge dataset from which we can calculate the 'true' ATE

true_EY1 = df['Y1'].mean()
true_EY0 = df['Y0'].mean()
true_ATE = true_EY1-true_EY0
print(f'True ATE = {true_ATE}')

True ATE = 0.17865750000000002


In [3]:
# now generate a much smaller dataset that we will actually use for our methods
df = generate_data_nonlinearity(10000, 556)
df

,w1,w2,w3,w4,A,Y,Y1,Y0
0,1.0,1.0,3.380,2.750,1.0,0.0,0.0,1.0
1,0.0,0.0,3.994,3.031,1.0,1.0,1.0,1.0
2,1.0,1.0,0.040,4.536,1.0,1.0,1.0,1.0
3,1.0,0.0,2.001,3.052,0.0,1.0,1.0,1.0
4,0.0,0.0,0.007,4.160,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
9995,0.0,1.0,1.777,2.746,1.0,1.0,1.0,1.0
9996,1.0,1.0,0.777,4.488,1.0,1.0,1.0,1.0
9997,1.0,1.0,1.428,2.478,1.0,1.0,1.0,0.0
9998,0.0,1.0,1.843,1.945,0.0,1.0,0.0,1.0


In [4]:
# we create a new dataset where the treatment variables are set to 1 and 0 

newdata_A1 = df.copy()
newdata_A1['A'] = 1

newdata_A0 = df.copy()
newdata_A0['A'] = 0

# TMLE  

### 1) Using a binary logistic regression model  

In [5]:
# first, we implement an intentionally wrong model (binary logistic regression, without the interaction terms)

# this is the outcome model, fit a binary logistic regression for it 
model = smf.logit("Y ~ A + w1 + w2 + w3 + w4", data=df)
model = model.fit()

# predict probabilities based on this data 
QAW = model.predict(df) # what does our model predict the outcome as (probability)
Q1W = model.predict(newdata_A1) # what if the patient had been treated
Q0W = model.predict(newdata_A0) # what if the patient had not been treated

# initial ATE estimate: 
init_ATE_est = Q1W.mean() - Q0W.mean() # difference in outcome if every patient treated vs every patient not treated
print(f'Initial (biased) ATE estimate = {init_ATE_est}')

# this is the treatment model (propensity score) 
ps_model = smf.logit("A ~ w1 + w2 + w3 + w4", data=df)
ps_model = ps_model.fit()

# use this model to calculate propensity scores 
gW = ps_model.predict(df)

target_model = targeting_step(df, gW, QAW) # carries out the targeting step to optimise the b-v tradeoff for the ATE
epsilon = target_model.params # coefficients in this targeting step

# use the epsilon values to improve the treatment model

# Convert Q0W and Q1W to logit scale and update them
logit_Q0W = np.log(Q0W / (1 - Q0W))
logit_Q1W = np.log(Q1W / (1 - Q1W))

# Update logit values with epsilon adjustments
logit_Q0W_1 = logit_Q0W + epsilon['H0W'] / (1 - gW)
logit_Q1W_1 = logit_Q1W + epsilon['H1W'] / gW

# Convert back to probabilities using inverse-logit
Q0W_1 = expit(logit_Q0W_1)
Q1W_1 = expit(logit_Q1W_1)

# Now we can calculate an improved ATE
EY1_tmle_1 = Q1W_1.mean()
EY0_tmle_1 = Q0W_1.mean()
ATE_tmle_1 = EY1_tmle_1 - EY0_tmle_1

Optimization terminated successfully.
         Current function value: 0.512408
         Iterations 6
Initial (biased) ATE estimate = 0.18770227170831688
Optimization terminated successfully.
         Current function value: 0.270700
         Iterations 8


### 2) Using a ML model  
  
Now we use ML models for the treatment and outcome models, which should be more robust to bias and model misspecification

In [6]:

# Define predictors and outcome
X = df[['A', 'w1', 'w2', 'w3', 'w4']]  # Covariates and treatment
y = df['Y']  # Outcome

# Initialize the Gradient Boosting model
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the model
gb_model.fit(X, y)

QAW = gb_model.predict_proba(X)[:,1]
Q1W = gb_model.predict_proba(newdata_A1[['A', 'w1', 'w2', 'w3', 'w4']])[:,1]
Q0W = gb_model.predict_proba(newdata_A0[['A', 'w1', 'w2', 'w3', 'w4']])[:,1]

ml_ATE_est_no_target = Q1W.mean() - Q0W.mean() # initial estimate of ATE using ML, but no targeting

# Define predictors and outcome
X = df[['w1', 'w2', 'w3', 'w4']]  # Covariates and treatment
y = df['A']  # Outcome

# Initialize the Gradient Boosting model
ps_model_ml = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fit the model
ps_model_ml.fit(X, y)

propensity_estimates_ml = ps_model_ml.predict_proba(X)[:,1]

target_model = targeting_step(df, propensity_estimates_ml, QAW) # carries out the targeting step to optimise the b-v tradeoff for the ATE
epsilon = target_model.params # coefficients in this targeting step

# use the epsilon values to improve the treatment model

# Convert Q0W and Q1W to logit scale and update them
logit_Q0W = np.log(Q0W / (1 - Q0W))
logit_Q1W = np.log(Q1W / (1 - Q1W))

# Update logit values with epsilon adjustments
logit_Q0W_1 = logit_Q0W + epsilon['H0W'] / (1 - propensity_estimates_ml)
logit_Q1W_1 = logit_Q1W + epsilon['H1W'] / propensity_estimates_ml

# Convert back to probabilities using inverse-logit
Q0W_1 = expit(logit_Q0W_1)
Q1W_1 = expit(logit_Q1W_1)

# Now we can calculate an improved ATE
EY1_tmle_2 = Q1W_1.mean()
EY0_tmle_2 = Q0W_1.mean()
ATE_tmle_2 = EY1_tmle_2 - EY0_tmle_2

### 3) Using stacked classifiers (superlearner)  
  
This approach uses an ensemble of ML methods, as recommended by the developers of the TMLE `R` package

In [7]:
# Define predictors and outcome
X = df[['A', 'w1', 'w2', 'w3', 'w4']]  # Covariates and treatment
y = df['Y']  # Outcome

# Define base models
base_models = [
    ('gradient_boosting', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),
    ('random_forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(probability=True, kernel='linear', random_state=42))
]

# Define meta-model
meta_model = LogisticRegression()

# Create the stacking classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Fit the stacking model
stacking_model.fit(X, y)

# Predict probabilities
QA = stacking_model.predict_proba(X)[:,1]
Q1 = stacking_model.predict_proba(newdata_A1[['A', 'w1', 'w2', 'w3', 'w4']])[:,1]
Q0 = stacking_model.predict_proba(newdata_A0[['A', 'w1', 'w2', 'w3', 'w4']])[:,1]

sl_ATE_est_no_target = Q1.mean() - Q0.mean() # initial estimate of ATE using superlearner, but no targeting

# Define predictors and outcome
X = df[['w1', 'w2', 'w3', 'w4']]  # Covariates and treatment
y = df['A']  # Outcome

# Define base models
base_models = [
    ('gradient_boosting', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),
    ('random_forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(probability=True, kernel='linear', random_state=42))
]

# Define meta-model
meta_model = LogisticRegression()

# Create the stacking classifier
stacking_model_ps = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Fit the stacking model
stacking_model_ps.fit(X, y)

propensity_estimates_sl = stacking_model_ps.predict_proba(X)[:,1]

target_model = targeting_step(df, propensity_estimates_sl, QA) # carries out the targeting step to optimise the b-v tradeoff for the ATE
epsilon = target_model.params # coefficients in this targeting step

# use the epsilon values to improve the treatment model

# Convert Q0W and Q1W to logit scale and update them
logit_Q0 = np.log(Q0 / (1 - Q0))
logit_Q1 = np.log(Q1 / (1 - Q1))

# Update logit values with epsilon adjustments
logit_Q0_1 = logit_Q0 + epsilon['H0W'] / (1 - propensity_estimates_sl)
logit_Q1_1 = logit_Q1 + epsilon['H1W'] / propensity_estimates_sl

# Convert back to probabilities using inverse-logit
Q0_1 = expit(logit_Q0_1)
Q1_1 = expit(logit_Q1_1)

# Now we can calculate an improved ATE
EY1_tmle_3 = Q1_1.mean()
EY0_tmle_3 = Q0_1.mean()
ATE_tmle_3 = EY1_tmle_3 - EY0_tmle_3


### Summarise Results

In [8]:
result_summary = pd.DataFrame({
    'True ATE': [true_ATE, 100*abs(true_ATE-true_ATE)/true_ATE], 
    'ATE (L.R)': [ATE_tmle_1, 100*abs(ATE_tmle_1-true_ATE)/true_ATE], 
    'ATE (ML)': [ATE_tmle_2, 100*abs(ATE_tmle_2-true_ATE)/true_ATE], 
    'ATE (Superlearning)': [ATE_tmle_3, 100*abs(ATE_tmle_3-true_ATE)/true_ATE] 
}, index=['Value', 'Difference from ATE (%)'])

In [9]:
print(result_summary)

                         True ATE  ATE (L.R)  ATE (ML)  ATE (Superlearning)
Value                    0.178658   0.182663  0.191273             0.176185
Difference from ATE (%)  0.000000   2.241931  7.061223             1.383896
